In [5]:
!pip install ffmpeg
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
import glob

In [7]:
import librosa
import numpy as np

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=22050)
    
    # Extract MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    print(f"MFCC Shape: {mfcc.shape}, MFCC Mean Shape: {mfcc_mean.shape}")
    
    # Extract Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
    print(f"Spectral Contrast Shape: {spectral_contrast.shape}, Spectral Contrast Mean Shape: {spectral_contrast_mean.shape}")
    
    # Extract Chroma Features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)
    print(f"Chroma Shape: {chroma.shape}, Chroma Mean Shape: {chroma_mean.shape}")
    
    # Extract Tempo (BPM)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    print(f"Tempo: {tempo} (Expected scalar)")
    tempo = tempo[0] if isinstance(tempo, np.ndarray) else tempo  # Ensure it's a scalar
    
    # Extract RMS (Root Mean Square Energy)
    rms = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms)
    print(f"RMS Shape: {rms.shape}, RMS Mean: {rms_mean} (Expected scalar)")
    
    # Ensure all features are 1D before stacking
    features = np.hstack([mfcc_mean, spectral_contrast_mean, chroma_mean, np.array([tempo]), np.array([rms_mean])])
    
    print(f"Feature vector shape: {features.shape}")
    return features


In [8]:
def load_data(dataset_path):
    features = []
    labels = []
    genres = os.listdir(dataset_path)  # Assume each genre is a subfolder

    for genre in genres:
        genre_path = os.path.join(dataset_path, genre)
        audio_files = glob.glob(os.path.join(genre_path, "*.wav"))  # Or "*.mp3"
        
        for file in audio_files:
            try:
                # Extract features for each song
                song_features = extract_features(file)
                features.append(song_features)
                labels.append(genre)
            except Exception as e:
                print(f"Error processing {file}: {e}")
    
    return np.array(features), np.array(labels)


In [10]:
# Path to your dataset directory containing genre subfolders
dataset_path = "C:/Users/khaja/Downloads/Data/genres_original"

# Extract features and labels
X, y = load_data(dataset_path)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

# Test the model
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.1301843225955963 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [107.66601562] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.09590782225131989 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [161.49902344] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.17547321319580078 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293),

C:\Users\khaja\AppData\Local\Temp\ipykernel_21236\2908354849.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=22050)
C:\Users\khaja\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing C:/Users/khaja/Downloads/Data/genres_original\jazz\jazz.00054.wav: 
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.09146158397197723 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1300), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1300), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1300), Chroma Mean Shape: (12,)
Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1300), RMS Mean: 0.19825299084186554 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [95.703125] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.07809878140687943 (Expected scalar)
Feature vector shape: (34,)
M

Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.14658696949481964 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.10329493135213852 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.090626560151577 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Sha

Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.15660792589187622 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.09581992775201797 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [135.99917763] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.08229553699493408 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [95.703125] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.17967274

...

Tempo: [99.38401442] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.17930187284946442 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [99.38401442] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.17930187284946442 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [129.19921875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.0567508339881897 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [99.38401442] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.114172205

Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.2607937455177307 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.12818020582199097 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [161.49902344] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.15342064201831818 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [117.45383523] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.12335520

Spectral Contrast Shape: (7, 1305), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1305), Chroma Mean Shape: (12,)
Tempo: [135.99917763] (Expected scalar)
RMS Shape: (1, 1305), RMS Mean: 0.21626079082489014 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [129.19921875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.11334493011236191 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1292), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1292), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1292), Chroma Mean Shape: (12,)
Tempo: [135.99917763] (Expected scalar)
RMS Shape: (1, 1292), RMS Mean: 0.21929600834846497 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma S

MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.05307672545313835 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [151.99908088] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.10706774890422821 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.11465876549482346 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), 

Tempo: [123.046875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.15463757514953613 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [143.5546875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.1082364171743393 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [129.19921875] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.10686670243740082 (Expected scalar)
Feature vector shape: (34,)
MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [103.359375] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.10572244971

In [13]:
# Function to predict the genre of a new song
def predict_genre(song_path, classifier):
    # Extract features from the new song
    song_features = extract_features(song_path).reshape(1, -1)
    
    # Predict genre using the trained classifier
    genre_pred = classifier.predict(song_features)
    
    return genre_pred[0]  # Return the predicted genre

# Example usage: Predict the genre of a new song
new_song_path = "C:/Users/khaja/Downloads/Data/genres_original/hiphop/hiphop.00004.wav"
predicted_genre = predict_genre(new_song_path, classifier)
print(f"The predicted genre is: {predicted_genre}")


MFCC Shape: (13, 1293), MFCC Mean Shape: (13,)
Spectral Contrast Shape: (7, 1293), Spectral Contrast Mean Shape: (7,)
Chroma Shape: (12, 1293), Chroma Mean Shape: (12,)
Tempo: [95.703125] (Expected scalar)
RMS Shape: (1, 1293), RMS Mean: 0.20046541094779968 (Expected scalar)
Feature vector shape: (34,)
The predicted genre is: hiphop
